<a href="https://colab.research.google.com/github/sethi-rasna/Wine_Os/blob/main/MACHINE_LEARNING_final_group_project_MACHINE_LEARNING_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Import our input dataset
application_df = pd.read_csv('resources/red_white.csv')
application_df.head()

,wine_type,winetype_numeric,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,fixed acidity.1,sulphates 1=low 2=high,alcohol 1=low 2=high,fixed acidity 1=low 2=high,healthiness,quality
0,red,1,5.0,1.02,0.04,1.4,0.045,41.0,85.0,0.99380,3.75,0.48,10.5,5.0,1,2,1,1,2
1,red,1,6.0,0.50,0.04,2.2,0.092,13.0,26.0,0.99647,3.46,0.47,10.0,6.0,1,2,1,1,2
2,red,1,6.1,0.32,0.25,1.8,0.086,5.0,32.0,0.99464,3.36,0.44,10.1,6.1,1,2,1,1,2
3,red,1,6.1,0.34,0.25,1.8,0.084,4.0,28.0,0.99464,3.36,0.44,10.1,6.1,1,2,1,1,2
4,red,1,6.1,0.64,0.02,2.4,0.069,26.0,46.0,0.99358,3.47,0.45,11.0,6.1,1,2,1,1,2


In [2]:
# Load the raw data to pgAdmin database
# Store environmental variable
from getpass import getpass
password = getpass('!What is your password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://Red_White_Wine/postgres@PostgreSQL 14"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

!What is your password········


In [23]:
import os
import psycopg2 as ps
conn = ps.connect(
    host=os.environ["postgres"],
    port=os.environ["5432"],
    dbname=os.environ["Red_White_Wine"],
    user=os.environ["postgres"],
    password=os.environ["!Hebob77"])

KeyError: 'postgres'

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
%matplotlib inline

In [4]:
#Drop the non-beneficial ID columns, 'volatile acidity','citric acid', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide'.
application_df = application_df.drop(['wine_type','volatile acidity','citric acid', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide'],1)
application_df.head(10)

C:\Users\sethi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,winetype_numeric,fixed acidity,residual sugar,density,pH,sulphates,alcohol,fixed acidity.1,sulphates 1=low 2=high,alcohol 1=low 2=high,fixed acidity 1=low 2=high,healthiness,quality
0,1,5.0,1.4,0.99380,3.75,0.48,10.5,5.0,1,2,1,1,2
1,1,6.0,2.2,0.99647,3.46,0.47,10.0,6.0,1,2,1,1,2
2,1,6.1,1.8,0.99464,3.36,0.44,10.1,6.1,1,2,1,1,2
3,1,6.1,1.8,0.99464,3.36,0.44,10.1,6.1,1,2,1,1,2
4,1,6.1,2.4,0.99358,3.47,0.45,11.0,6.1,1,2,1,1,2
5,1,6.3,2.0,0.99488,3.60,0.46,11.2,6.3,1,2,1,1,2
6,1,6.3,1.4,0.99220,3.45,0.48,12.3,6.3,1,2,1,1,2
7,1,6.4,2.1,0.99490,3.49,0.49,11.4,6.4,1,2,1,1,2
8,1,6.4,3.2,0.99500,3.61,0.49,12.7,6.4,1,2,1,1,2
9,1,6.6,1.6,0.99396,3.33,0.37,10.4,6.6,1,2,1,1,2


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

winetype_numeric                2
fixed acidity                 102
residual sugar                254
density                       752
pH                             98
sulphates                     108
alcohol                        84
fixed acidity.1               102
sulphates 1=low 2=high          2
alcohol 1=low 2=high            2
fixed acidity 1=low 2=high      2
healthiness                     2
quality                         2
dtype: int64

In [6]:
# Look at winetype_numeric value counts for binning
application_count = application_df.winetype_numeric.value_counts()
application_count


1    1470
2    1470
Name: winetype_numeric, dtype: int64

In [7]:
# Generate our categorical variable lists 
#application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
#application_df[application_cat].nunique()

In [8]:
# define "healthy" column with metrics scores based on sulphate content. IF low then 1= HEALTHY If high then 2= NOT HEALTHY 

y = application_df[['healthiness','quality']].values
X = application_df.drop(['healthiness','quality'],1).values 

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)



C:\Users\sethi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=2,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                960       
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 2)                 62        
                                                                 
Total params: 3,452
Trainable params: 3,452
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
69/69 [==============================] - 3s 7ms/step - loss: -1.1641 - accuracy: 0.3887
Epoch 2/100
69/69 [==============================] - 0s 7ms/step - loss: -14.8764 - accuracy: 0.2553
Epoch 3/100
69/69 [==============================] - 0s 7ms/step - loss: -77.2996 - accuracy: 0.1937
Epoch 4/100
69/69 [==============================] - 0s 7ms/step - loss: -251.1166 - accuracy: 0.1937
Epoch 5/100
69/69 [==============================] - 0s 7ms/step - loss: -599.8790 - accuracy: 0.1937
Epoch 6/100
69/69 [==============================] - 0s 7ms/step - loss: -1182.6370 - accuracy: 0.1937
Epoch 7/100
69/69 [==============================] - 0s 7ms/step - loss: -2053.6960 - accuracy: 0.1937
Epoch 8/100
69/69 [==============================] - 0s 6ms/step - loss: -3260.8010 - accuracy: 0.1937
Epoch 9/100
69/69 [==============================] - 0s 6ms/step - loss: -4838.9111 - accuracy: 0.1937
Epoch 10/100
69/69 [==============================] - 0s 7ms/step - loss: -6818.32

69/69 [==============================] - 0s 6ms/step - loss: -2220217.7500 - accuracy: 0.1937
Epoch 79/100
69/69 [==============================] - 0s 7ms/step - loss: -2292018.0000 - accuracy: 0.1937
Epoch 80/100
69/69 [==============================] - 0s 6ms/step - loss: -2365354.7500 - accuracy: 0.1937
Epoch 81/100
69/69 [==============================] - 0s 7ms/step - loss: -2439889.2500 - accuracy: 0.1937
Epoch 82/100
69/69 [==============================] - 0s 6ms/step - loss: -2515960.7500 - accuracy: 0.1937
Epoch 83/100
69/69 [==============================] - 0s 7ms/step - loss: -2593156.7500 - accuracy: 0.1937
Epoch 84/100
69/69 [==============================] - 0s 7ms/step - loss: -2672046.5000 - accuracy: 0.1937
Epoch 85/100
69/69 [==============================] - 0s 7ms/step - loss: -2752137.0000 - accuracy: 0.1937
Epoch 86/100
69/69 [==============================] - 0s 6ms/step - loss: -2833710.0000 - accuracy: 0.1937
Epoch 87/100
69/69 [==============================

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

23/23 - 0s - loss: -4.1752e+06 - accuracy: 0.8054 - 148ms/epoch - 6ms/step
Loss: -4175249.25, Accuracy: 0.8054421544075012


In [43]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization1/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization1/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
69/69 [==============================] - 0s 1ms/step - loss: -4258695.0000 - accuracy: 0.8063
Epoch 2/100
 1/69 [..............................] - ETA: 0s - loss: -3622044.0000 - accuracy: 0.8438
Epoch 2: saving model to checkpoints/Optimization1/weights.02.hdf5
69/69 [==============================] - 0s 1ms/step - loss: -4343075.0000 - accuracy: 0.8063
Epoch 3/100
35/69 [==============>...............] - ETA: 0s - loss: -4353378.0000 - accuracy: 0.8107
Epoch 3: saving model to checkpoints/Optimization1/weights.03.hdf5
69/69 [==============================] - 0s 2ms/step - loss: -4430429.0000 - accuracy: 0.8063
Epoch 4/100
69/69 [==============================] - 0s 1ms/step - loss: -4519290.0000 - accuracy: 0.8063
Epoch 5/100
 1/69 [..............................] - ETA: 0s - loss: -3445044.0000 - accuracy: 0.7812
Epoch 5: saving model to checkpoints/Optimization1/weights.05.hdf5
69/69 [==============================] - 0s 1ms/step - loss: -4610162.0000 - accuracy: 0.8063

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

23/23 - 1s - loss: -4.1633e+06 - accuracy: 0.1946 - 727ms/epoch - 32ms/step
Loss: -4163334.5, Accuracy: 0.1945578157901764


In [14]:
# Export our model to HDF5 file
nn.save('groupprojectMachineLearning_model')


INFO:tensorflow:Assets written to: groupprojectMachineLearning_model\assets
